# **SIA Check (serialized execution)**

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from tqdm.notebook import tqdm

## **Definition of Methods for Auctions**

In [ ]:
class Buyer:
    def __init__(self, buyer_id, distances, pv=1000, beta=0.5, pvmax=1000):
        self.id = buyer_id
        self.state = "overbid"
        self.pv = pv
        self.beta = beta
        self.pvmax = pvmax
        self.distances = distances
        self.prices = {}
        
    def inform_price(self, auction_id, price):
        self.prices[auction_id] = price if price <= self.pv else np.inf
        if np.all(np.array(self.prices) == np.inf):
            self.state = "out_of_budget"
        
    def compute_costs(self):
        prices, distances, id_list = [], [], []
        for p in self.distances:
            prices.append(self.prices[p])
            distances.append(self.distances[p])
            id_list.append(p)
        prices = np.array(prices)
        distances = np.array(distances)
        cost = self.beta*prices/self.pvmax + (1-self.beta)*distances/np.max(distances)
        return cost, id_list
        
    def ask_bid(self, auction_id, price):
        bids = False
        if (self.state == "overbid") and (price <= self.pv):
            costs, ids = self.compute_costs()
            prefs = np.where(costs == costs.min())[0]
            pref_ids = []
            for p in prefs:
                pref_ids.append(ids[p])
            bids = auction_id in pref_ids
        elif price > self.pv:
            self.prices[auction_id] = np.inf
        if bids:
            self.state = "winning"
        return bids
    
    def tell_overbid(self):
        self.state = "overbid"
        
    def tell_won(self):
        self.state = "won"

In [ ]:
class Auction:
    def __init__(self, id_, starting_price, epsilon=1.0):
        self.id = id_
        self.price = starting_price
        self.buyers = []
        self.epsilon = epsilon
        self.winner = None
        
    def add_buyer(self, buyer):
        self.buyers.append(buyer)
        
    def _inform_buyers(self):
        for b in self.buyers:
            b.inform_price(self.id, self.price)
        
    def auction_round(self):
        bid_received = False
        for b in self.buyers:
            bids = b.ask_bid(self.id, self.price)
            if bids:
                if self.winner is not None:
                    self.winner.tell_overbid()
                self.winner = b
                self.price += self.epsilon
                self._inform_buyers()
                bid_received = True
        return bid_received
                
    def terminate(self):
        winner_id = ""
        if self.winner is not None:
            self.winner.tell_won()
            winner_id = self.winner.id
        return {"winner": winner_id,
                "price" : self.price-self.epsilon}

In [ ]:
def check_terminable(buyers):
    non_terminated_buyers = 0
    for b in buyers:
        if (b.state != "winning") and (b.state != "out_of_budget"):
            non_terminated_buyers += 1
    return len(buyers) - non_terminated_buyers

def run_auctions_with_prices(auctions, buyers, r_max):
    auction_results = {}
    auction_prices = []
    rs = np.zeros(len(auctions))
    
    pbar = tqdm(total=len(buyers))
    terminables = 0
    while np.sum(rs<r_max) and (terminables != len(buyers)):
        ap_ = {}
        for a in auctions:
            ap_[a.id] = a.price
        auction_prices.append(ap_)
        
        pbar.n = terminables
        pbar.refresh()
        for i, a in enumerate(auctions):
            bid_received = a.auction_round()
            if bid_received:
                rs[i] = 0
            else:
                rs[i] += 1
        terminables = check_terminable(buyers)
                
    pbar.n = len(auctions)
    pbar.refresh()
    for a in auctions:
        auction_results[a.id] = a.terminate()  
    pbar.close()
    return auction_results, auction_prices

def run_auctions(auctions, buyers, r_max):
    auction_results = {}
    rs = np.zeros(len(auctions))
    
    pbar = tqdm(total=len(auctions))
    terminables = 0
    while np.sum(rs<r_max) and (terminables != len(buyers)):
        pbar.n = terminables
        pbar.refresh()
        for i, a in enumerate(auctions):
            bid_received = a.auction_round()
            if bid_received:
                rs[i] = 0
            else:
                rs[i] += 1
        terminables = check_terminable(buyers)
                
    pbar.n = len(auctions)
    pbar.refresh()
    for a in auctions:
        auction_results[a.id] = a.terminate()  
    pbar.close()
    return auction_results

In [ ]:
def get_won_auctions(auction_results, buyers):
    won_auctions = {}
    for b in buyers:
        for a in auction_results:
            if auction_results[a]["winner"] == b.id:
                won_auctions[b.id] = a
    return won_auctions

In [ ]:
import matplotlib.pyplot as plt

def plot_dist_price_results(auction_results, buyers):
    won_auctions = get_won_auctions(auction_results, buyers)
    distances, prices = [],[]
    
    for b in buyers:
        if b.id in won_auctions:
            a_i = won_auctions[b.id]
            distances.append(b.distances[a_i])
            prices.append(auction_results[a_i]["price"])
            
    fig, ax = plt.subplots(1,1, figsize=(6,4))
    ax.scatter(distances, prices, alpha=0.66)
    ax.set_xlabel("distance [m]")
    ax.set_ylabel("price [HUF]")
    return fig, ax

def plot_position_price_results(auction_results, buyers, positions, label=""):
    won_auctions = get_won_auctions(auction_results, buyers)
    distances, prices = [],[]
    
    for b in buyers:
        if b.id in won_auctions:
            a_i = won_auctions[b.id]
            idx = int(a_i.split("_")[-1])
            distances.append(positions[idx])
            prices.append(auction_results[a_i]["price"])
            
    fig, ax = plt.subplots(1,1, figsize=(6,3))
    ax.scatter(distances, prices, color="k", alpha=0.66, label=label)
    ax.set_xlabel("position [m]")
    ax.set_ylabel("price [¢]")
    return fig, ax

def plot_utility(auction_results, buyers, betas, pvmax=1000):
    won_auctions = get_won_auctions(auction_results, buyers)
    distances, prices = [],[]
    
    for b in buyers:
        if b.id in won_auctions:
            a_i = won_auctions[b.id]
            distances.append(b.distances[a_i])
            prices.append(auction_results[a_i]["price"])
            
    distances = np.array(distances)
    prices = np.array(prices)
    betas = np.array(betas)
    
    utilities = 1 - (betas*prices/(pvmax) + (1-betas)*distances/max(distances))
            
    fig, ax = plt.subplots(1,1, figsize=(6,4))
    ax.scatter(distances, utilities, alpha=0.66)
    ax.set_xlabel("distance [m]")
    ax.set_ylabel("utilities")
    return fig, ax

## **Random setup**

### $\beta=0.5$

In [ ]:
prices = np.random.randint(0, 400, 100)

buyers = []
for b_i in range(100):
    distance_ = {}
    for a_i in range(100):
        distance_[f"a_{a_i}"] = np.random.randint(0, 5000)
    buyers.append(Buyer(f"b_{b_i}", distance_))
auctions = [Auction(f"a_{i}", prices[i]) for i in range(100)]
for a in auctions:
    for b in buyers:
        a.add_buyer(b)
        b.inform_price(a.id, a.price)

In [ ]:
results = run_auctions(auctions, buyers, 1001)

In [ ]:
fig, ax = plot_dist_price_results(results, buyers)
fig.show()

In [ ]:
plot_utility(results, buyers, [0.5 for _ in range(100)])

### $\beta = 0.1$

In [ ]:
prices = np.random.randint(0, 400, 100)

buyers = []
for b_i in range(100):
    distance_ = {}
    for a_i in range(100):
        distance_[f"a_{a_i}"] = np.random.randint(0, 5000)
    buyers.append(Buyer(f"b_{b_i}", distance_, beta=0.1))
auctions = [Auction(f"a_{i}", prices[i]) for i in range(100)]
for a in auctions:
    for b in buyers:
        a.add_buyer(b)
        b.inform_price(a.id, a.price)

In [ ]:
%%time

results = run_auctions(auctions, buyers, 1001)

In [ ]:
fig, ax = plot_dist_price_results(results, buyers)
fig.show()

In [ ]:
plot_utility(results, buyers, [0.1 for _ in range(100)])

### $\beta = 0.9$

In [ ]:
prices = np.random.randint(0, 400, 100)

buyers = []
for b_i in range(100):
    distance_ = {}
    for a_i in range(100):
        distance_[f"a_{a_i}"] = np.random.randint(0, 5000)
    buyers.append(Buyer(f"b_{b_i}", distance_, beta=0.9))
auctions = [Auction(f"a_{i}", prices[i]) for i in range(100)]
for a in auctions:
    for b in buyers:
        a.add_buyer(b)
        b.inform_price(a.id, a.price)

In [ ]:
%%time

results = run_auctions(auctions, buyers, 1001)

In [ ]:
fig, ax = plot_dist_price_results(results, buyers)
fig.show()

In [ ]:
plot_utility(results, buyers, [0.9 for _ in range(100)])

## **CBD-setup**

In [ ]:
prices = []
positions = []
for i in range(10):
    prices += [500-i*50]*10
    positions += [i*100]*10
    
prices = np.array(prices)
positions = np.array(positions)

In [ ]:
plt.scatter(positions, prices)
plt.xlabel("distance [m]")
plt.ylabel("price [HUF]")

### 0-centered demand

#### $\beta=0.5$

In [ ]:
buyers = []
for b_i in range(100):
    distance_ = {}
    for a_i in range(100):
        distance_[f"a_{a_i}"] = positions[a_i]
    buyers.append(Buyer(f"b_{b_i}", distance_, beta=0.5))
auctions = [Auction(f"a_{i}", prices[i]) for i in range(100)]
for a in auctions:
    for b in buyers:
        a.add_buyer(b)
        b.inform_price(a.id, a.price)

In [ ]:
%%time

results = run_auctions(auctions, buyers, 1001*100)

In [ ]:
fig, ax = plot_dist_price_results(results, buyers)
fig.show()

In [ ]:
plot_utility(results, buyers, [0.5 for _ in range(100)])

#### $\beta = 0.1$

In [ ]:
buyers = []
for b_i in range(100):
    distance_ = {}
    for a_i in range(100):
        distance_[f"a_{a_i}"] = positions[a_i]
    buyers.append(Buyer(f"b_{b_i}", distance_, beta=0.1))
auctions = [Auction(f"a_{i}", prices[i], epsilon=10) for i in range(100)]
for a in auctions:
    for b in buyers:
        a.add_buyer(b)
        b.inform_price(a.id, a.price)

In [ ]:
results = run_auctions(auctions, buyers, 1001*100)

In [ ]:
fig, ax = plot_dist_price_results(results, buyers)
fig.show()

In [ ]:
plot_utility(results, buyers, [0.1 for _ in range(100)])

#### $\beta = 0.9$

In [ ]:
buyers = []
for b_i in range(100):
    distance_ = {}
    for a_i in range(100):
        distance_[f"a_{a_i}"] = positions[a_i]
    buyers.append(Buyer(f"b_{b_i}", distance_, beta=0.9))
auctions = [Auction(f"a_{i}", prices[i], epsilon=10) for i in range(100)]
for a in auctions:
    for b in buyers:
        a.add_buyer(b)
        b.inform_price(a.id, a.price)

In [ ]:
results = run_auctions(auctions, buyers, 1001*100)

In [ ]:
fig, ax = plot_dist_price_results(results, buyers)
fig.show()

In [ ]:
plot_utility(results, buyers, [0.1 for _ in range(100)])

#### Mostly small $\beta$

In [ ]:
buyers = []
for b_i in range(100):
    distance_ = {}
    for a_i in range(100):
        distance_[f"a_{a_i}"] = positions[a_i]
    buyers.append(Buyer(f"b_{b_i}", distance_, beta=0.1 if b_i<67 else 0.9))
auctions = [Auction(f"a_{i}", prices[i], epsilon=10) for i in range(100)]
for a in auctions:
    for b in buyers:
        a.add_buyer(b)
        b.inform_price(a.id, a.price)

In [ ]:
results = run_auctions(auctions, buyers, 1001*100)

In [ ]:
fig, ax = plot_dist_price_results(results, buyers)
fig.show()

In [ ]:
plot_utility(results, buyers, [0.1 for _ in range(67)]+[0.9 for _ in range(33)])

### **Random demand**

#### $\beta = 0.5$

In [ ]:
buyers = []
for b_i in range(100):
    distance_ = {}
    buyer_dest = np.random.uniform(0, 1000, 1)[0]
    for a_i in range(100):
        distance_[f"a_{a_i}"] = np.abs(buyer_dest - positions[a_i])
    buyers.append(Buyer(f"b_{b_i}", distance_, beta=0.5))
auctions = [Auction(f"a_{i}", prices[i]) for i in range(100)]
for a in auctions:
    for b in buyers:
        a.add_buyer(b)
        b.inform_price(a.id, a.price)

In [ ]:
results = run_auctions(auctions, buyers, 1001*100)

In [ ]:
fig, ax = plot_dist_price_results(results, buyers)
fig.show()

In [ ]:
plot_utility(results, buyers, [0.5 for _ in range(100)])

#### $\beta = 0.1$

In [ ]:
buyers = []
for b_i in range(100):
    distance_ = {}
    buyer_dest = np.random.uniform(0, 1000, 1)[0]
    for a_i in range(100):
        distance_[f"a_{a_i}"] = np.abs(buyer_dest - positions[a_i])
    buyers.append(Buyer(f"b_{b_i}", distance_, beta=0.1))
auctions = [Auction(f"a_{i}", prices[i]) for i in range(100)]
for a in auctions:
    for b in buyers:
        a.add_buyer(b)
        b.inform_price(a.id, a.price)

In [ ]:
results = run_auctions(auctions, buyers, 1001*100)

In [ ]:
fig, ax = plot_dist_price_results(results, buyers)
fig.show()

In [ ]:
plot_utility(results, buyers, [0.1 for _ in range(100)])

#### $\beta = 0.9$

In [ ]:
buyers = []
for b_i in range(100):
    distance_ = {}
    buyer_dest = np.random.uniform(0, 1000, 1)[0]
    for a_i in range(100):
        distance_[f"a_{a_i}"] = np.abs(buyer_dest - positions[a_i])
    buyers.append(Buyer(f"b_{b_i}", distance_, beta=0.9))
auctions = [Auction(f"a_{i}", prices[i]) for i in range(100)]
for a in auctions:
    for b in buyers:
        a.add_buyer(b)
        b.inform_price(a.id, a.price)

In [ ]:
results = run_auctions(auctions, buyers, 1001*100)

In [ ]:
fig, ax = plot_dist_price_results(results, buyers)
fig.show()

In [ ]:
plot_utility(results, buyers, [0.9 for _ in range(100)])

## **Various tests**

### **Underdemand, random setup, $\beta = 1.0$**

In [ ]:
prices = np.random.randint(0, 400, 100)

buyers = []
for b_i in range(30):
    distance_ = {}
    for a_i in range(100):
        distance_[f"a_{a_i}"] = np.random.randint(0, 5000)
    buyers.append(Buyer(f"b_{b_i}", distance_, beta=1.0))
auctions = [Auction(f"a_{i}", prices[i]) for i in range(100)]
for a in auctions:
    for b in buyers:
        a.add_buyer(b)
        b.inform_price(a.id, a.price)

In [ ]:
prices[np.argsort(prices)[29]]

In [ ]:
results = run_auctions(auctions, buyers, 1001*100)

In [ ]:
fig, ax = plot_dist_price_results(results, buyers)
fig.show()

In [ ]:
plot_utility(results, buyers, [0.5 for _ in range(30)])

### **Underdemand, 0-centered demand, $\beta=1.0$**

In [ ]:
prices = []
positions = []
for i in range(10):
    prices += [500-i*50]*10
    positions += [i*100]*10
    
prices = np.array(prices)
positions = np.array(positions)

In [ ]:
buyers = []
for b_i in range(30):
    distance_ = {}
    for a_i in range(100):
        distance_[f"a_{a_i}"] = positions[a_i]
    buyers.append(Buyer(f"b_{b_i}", distance_, beta=1.0))
auctions = [Auction(f"a_{i}", prices[i]) for i in range(100)]
for a in auctions:
    for b in buyers:
        a.add_buyer(b)
        b.inform_price(a.id, a.price)

In [ ]:
prices[np.argsort(prices)[29]]

In [ ]:
results = run_auctions(auctions, buyers, 1001*100)

In [ ]:
fig, ax = plot_dist_price_results(results, buyers)
fig.show()

In [ ]:
plot_utility(results, buyers, [0.5 for _ in range(30)])

### **Parking sizing**

If parking spaces are assigned by SIA, then what parking size maximizes the income of the parking lot operator?

In [ ]:
M = 25 #M vehicles

In [ ]:
from tqdm.notebook import trange

total_incomes = []
distances = 50
prices = 500

for n in trange(1, 40): #n auctions
    auctions = []
    buyers = []
    
    #prepare auctions:
    for b_i in range(M):
        distance_ = {}
        for a_i in range(n):
            distance_[f"a_{a_i}"] = distances
        buyers.append(Buyer(f"b_{b_i}", distance_, beta=1.0))
    auctions = [Auction(f"a_{i}", prices) for i in range(n)]
    for a in auctions:
        for b in buyers:
            a.add_buyer(b)
            b.inform_price(a.id, a.price)

    #run auctions:
    results = run_auctions(auctions, buyers, 1001*100)
    
    p_ = 0
    for r in results:
        if results[r]["winner"] != "":
            p_ += results[r]["price"]
    total_incomes.append(p_)

In [ ]:
in_euro = np.array(total_incomes)/100

plt.figure(figsize=(6, 3))
plt.fill_between(np.linspace(1, 24, 24), in_euro[:24], alpha=0.15, hatch="\\\\", color="k", label="Theorem 3.")
plt.plot(np.linspace(1, 39, 39), in_euro, alpha=0.6, color="k")
plt.scatter(np.linspace(1, 24, 24), in_euro[:24], marker="o", color="k", )
plt.fill_between(np.linspace(25, 39, 15), in_euro[24:], alpha=0.3, hatch="//", color="k", label="Theorem 2.")
plt.scatter(np.linspace(25, 39, 15), in_euro[24:], marker="o", color="k")
plt.legend()
plt.ylabel("total income [€]")
plt.xlabel("#parkings")
plt.vlines(x=np.argmax(in_euro)+1, ymin = 0, ymax = max(in_euro)*1.1,
           color="k", linestyle="--", linewidth=3, alpha=0.66)

### Relevant and Irrelevant Parkings

#### Descending pricing, $\beta = 0.25$, overdemand

In [ ]:
distances = []
prices = []
for i in range(20):
    distances.append(i*25)
for i in range(80):
    distances.append(2000)

for i in range(20):
    prices.append(500-i*10)
for i in range(80):
    prices.append(50)

In [ ]:
plt.scatter(distances, prices)
plt.xlabel("distance [m]")
plt.ylabel("price [HUF]")

In [ ]:
buyers = []
for b_i in range(25):
    distance_ = {}
    for a_i in range(100):
        distance_[f"a_{a_i}"] = distances[a_i]
    buyers.append(Buyer(f"b_{b_i}", distance_, beta=0.25))
auctions = [Auction(f"a_{i}", prices[i]) for i in range(100)]
for a in auctions:
    for b in buyers:
        a.add_buyer(b)
        b.inform_price(a.id, a.price)

In [ ]:
results = run_auctions(auctions, buyers, 100*1001)

In [ ]:
plot_dist_price_results(results, buyers)

In [ ]:
plot_utility(results, buyers, [0.25 for _ in range(25)])

#### Descending pricing, $\beta = 0.25$, underdemand

In [ ]:
buyers = []
for b_i in range(15):
    distance_ = {}
    for a_i in range(100):
        distance_[f"a_{a_i}"] = distances[a_i]
    buyers.append(Buyer(f"b_{b_i}", distance_, beta=0.25))
auctions = [Auction(f"a_{i}", prices[i]) for i in range(100)]
for a in auctions:
    for b in buyers:
        a.add_buyer(b)
        b.inform_price(a.id, a.price)

In [ ]:
results = run_auctions(auctions, buyers, 100*1001)

In [ ]:
plot_dist_price_results(results, buyers)

In [ ]:
plot_utility(results, buyers, [0.25 for _ in range(15)])

#### Descending pricing, $\beta = 0.75$, underdemand

In [ ]:
buyers = []
for b_i in range(15):
    distance_ = {}
    for a_i in range(100):
        distance_[f"a_{a_i}"] = distances[a_i]
    buyers.append(Buyer(f"b_{b_i}", distance_, beta=0.75))
auctions = [Auction(f"a_{i}", prices[i]) for i in range(100)]
for a in auctions:
    for b in buyers:
        a.add_buyer(b)
        b.inform_price(a.id, a.price)

In [ ]:
results = run_auctions(auctions, buyers, 100*1001)

In [ ]:
plot_dist_price_results(results, buyers)

In [ ]:
plot_utility(results, buyers, [0.75 for _ in range(15)])

### **Prices during auctions**

#### Underdemand, random setup, $\beta=1.0$

In [ ]:
prices = np.random.randint(100, 400, 100)

buyers = []
for b_i in range(8):
    distance_ = {}
    for a_i in range(100):
        distance_[f"a_{a_i}"] = np.random.randint(0, 5000)
    buyers.append(Buyer(f"b_{b_i}", distance_, beta=1.0))
auctions = [Auction(f"a_{i}", prices[i]) for i in range(100)]
for a in auctions:
    for b in buyers:
        a.add_buyer(b)
        b.inform_price(a.id, a.price)

In [ ]:
results, auction_prices = run_auctions_with_prices(auctions, buyers, 100000)

In [ ]:
auctions_per_time = {}
for a in auctions:
    if results[a.id]["winner"] != "":
        ap_ = []
        for t in range(len(auction_prices)):
            ap_.append(auction_prices[t][a.id])
        auctions_per_time[a.id] = ap_

In [ ]:
for a in auctions:
    if a.id in auctions_per_time:
        plt.plot(auctions_per_time[a.id], label=a.id)
        
plt.legend(ncols=2)
plt.xlabel("auction rounds")
plt.ylabel("prices [HUF]")

#### Underdemand, small CBD setup $\beta = 1.0$

In [ ]:
distances = np.linspace(0, 18, 10)
prices = np.linspace(20, 2, 10)

In [ ]:
plt.scatter(distances, prices, alpha=.67)
plt.xlabel("distance")
plt.ylabel("price")

In [ ]:
buyers = []
for b_i in range(8):
    distance_ = {}
    for a_i in range(10):
        distance_[f"$A_{a_i}$"] = distances[a_i]
    buyers.append(Buyer(f"b_{b_i}", distance_, beta=1.0))
auctions = [Auction(f"$A_{i}$", prices[i], epsilon=1) for i in range(10)]
for a in auctions:
    for b in buyers:
        a.add_buyer(b)
        b.inform_price(a.id, a.price)

In [ ]:
results, auction_prices = run_auctions_with_prices(auctions, buyers, 100000)

In [ ]:
auctions_per_time = {}
for a in auctions:
    if results[a.id]["winner"] != "":
        ap_ = []
        for t in range(len(auction_prices)):
            ap_.append(auction_prices[t][a.id])
        auctions_per_time[a.id] = ap_

In [ ]:
plt.figure(figsize=(6,3))

i = 0
for a in auctions:
    if a.id in auctions_per_time:
        if i%2 == 0:
            marker = "x"
        else:
            marker = "o"
        plt.plot(auctions_per_time[a.id], label=a.id, marker=marker,
                 color="k", alpha=0.9-(i*0.1))
        i += 1
        
plt.legend(title="auctions", ncols=2)
plt.xlabel("auction rounds")
plt.yticks([0, 2, 4, 6, 8, 10, 12, 14, 16, 18])
plt.ylabel("prices [¢]")
plt.ylim(0, 18)

#### Overdemand, small CBD setup, $\beta=1.0$

In [ ]:
buyers = []
for b_i in range(11):
    distance_ = {}
    for a_i in range(10):
        distance_[f"a_{a_i}"] = distances[a_i]
    buyers.append(Buyer(f"b_{b_i}", distance_, beta=1.0, pv=30, pvmax=30))
auctions = [Auction(f"a_{i}", prices[i], epsilon=1) for i in range(10)]
for a in auctions:
    for b in buyers:
        a.add_buyer(b)
        b.inform_price(a.id, a.price)

In [ ]:
results, auction_prices = run_auctions_with_prices(auctions, buyers, 100000)

In [ ]:
auctions_per_time = {}
for a in auctions:
    if results[a.id]["winner"] != "":
        ap_ = []
        for t in range(len(auction_prices)):
            ap_.append(auction_prices[t][a.id])
        auctions_per_time[a.id] = ap_

In [ ]:
for a in auctions:
    if a.id in auctions_per_time:
        plt.plot(auctions_per_time[a.id], label=a.id)
        
plt.legend(ncols=2)
plt.xlabel("auction rounds")
plt.ylabel("prices [HUF]")
plt.xlim(0, 30)

### **Test Prices, $\beta = 0.1$**

#### Single peak in demand

In [ ]:
distances = []
for d in range(0, 5000, 50):
    for k in range(5):
        distances.append(d)
prices = [0]*len(distances)
demand = np.concatenate((np.random.normal(2000, 100, 100),
                         np.random.uniform(0, 5000, 100)))

In [ ]:
plt.hist(demand, bins=50, alpha=0.67)
plt.xlabel("demand")
plt.ylabel("position [m]")

In [ ]:
buyers = []
for b_i in range(200):
    distance_ = {}
    for a_i in range(500):
        distance_[f"a_{a_i}"] = np.abs(demand[b_i]-distances[a_i])
    buyers.append(Buyer(f"b_{b_i}", distance_, beta=0.1))
auctions = [Auction(f"a_{i}", prices[i]) for i in range(500)]
for a in auctions:
    for b in buyers:
        a.add_buyer(b)
        b.inform_price(a.id, a.price)

In [ ]:
results = run_auctions(auctions, buyers, 100*1001)

In [ ]:
fig, ax = plot_position_price_results(results, buyers, distances, label="auctioned prices")
bx = ax.twinx()
bx.hist(demand, bins=50, alpha=0.3, color="k", label="demand")
bx.set_yticks([])
#bx.legend(loc=0)
#ax.legend()
fig.legend(loc="upper right", bbox_to_anchor=(.9,.85))
#plt.ylabel("position [m]")

In [ ]:
import json
with open("../03_results/single_peak.json", "w") as f:
    json.dump(results, f)

#### Double peak demand

In [ ]:
distances = []
for d in range(0, 5000, 50):
    for k in range(5):
        distances.append(d)
prices = [0]*len(distances)
demand = np.concatenate((np.random.normal(1200, 100, 70),
                         np.random.normal(3600, 100, 30),
                         np.random.uniform(0, 5000, 100)))

In [ ]:
plt.hist(demand, bins=50, alpha=0.67)
plt.xlabel("demand")
plt.ylabel("position [m]")

In [ ]:
buyers = []
for b_i in range(200):
    distance_ = {}
    for a_i in range(500):
        distance_[f"a_{a_i}"] = np.abs(demand[b_i]-distances[a_i])
    buyers.append(Buyer(f"b_{b_i}", distance_, beta=0.1))
auctions = [Auction(f"a_{i}", prices[i]) for i in range(500)]
for a in auctions:
    for b in buyers:
        a.add_buyer(b)
        b.inform_price(a.id, a.price)

In [ ]:
results = run_auctions(auctions, buyers, 100*1001)

In [ ]:
fig, ax = plot_position_price_results(results, buyers, distances, label="auctioned prices")
bx = ax.twinx()
bx.hist(demand, bins=50, alpha=0.3, color="k", label="demand")
bx.set_yticks([])
#bx.legend(loc=0)
#ax.legend()
fig.legend(loc="upper right", bbox_to_anchor=(.9,.85))
#plt.ylabel("position [m]")

In [ ]:
import json
with open("../03_results/double_peak.json", "w") as f:
    json.dump(results, f)

#### Pedestrian zone

In [ ]:
distances = []
for d in range(0, 5000, 50):
    if (d<3000) or (d>4000):
        for k in range(6):
            distances.append(d)
prices = [0]*len(distances)
demand = np.random.uniform(0, 5000, 200)

In [ ]:
fig, ax = plt.subplots(1,1, figsize=(6,4))
ax.hist(demand, bins=50, alpha=0.67)
ax.set_xlabel("demand")
bx = ax.twinx()
bx.scatter(distances, prices, marker="x", color="tab:red", label="parkings")
bx.set_ylim(-0.05, 0.3)
bx.set_yticks([])
ax.set_ylabel("position [m]")
fig.legend()

In [ ]:
buyers = []
for b_i in range(200):
    distance_ = {}
    for a_i in range(len(distances)):
        distance_[f"a_{a_i}"] = np.abs(demand[b_i]-distances[a_i])
    buyers.append(Buyer(f"b_{b_i}", distance_, beta=0.1))
auctions = [Auction(f"a_{i}", prices[i]) for i in range(len(distances))]
for a in auctions:
    for b in buyers:
        a.add_buyer(b)
        b.inform_price(a.id, a.price)

In [ ]:
results = run_auctions(auctions, buyers, 100*1001)

In [ ]:
fig, ax = plot_position_price_results(results, buyers, distances, label="auctioned prices")
fig.set_figheight(4.3)
bx = ax.twinx()
bx.hist(demand, bins=50, alpha=0.2, color="k", label="demand")
bx.set_yticks([])
fig.legend(loc="upper right",)# bbox_to_anchor=(.9,.85))

In [ ]:
import json
with open("../03_results/ped_zone.json", "w") as f:
    json.dump(results, f)

#### $\beta = 0.99$, various $p_v$s

In [ ]:
distances = np.linspace(0, 16, 9)
prices = np.linspace(20, 4, 9)

plt.scatter(distances, prices, alpha=.66)
plt.xlabel("distance")
plt.ylabel("price")

In [ ]:
buyers = []
for b_i in range(12):
    distance_ = {}
    for a_i in range(len(distances)):
        distance_[f"a_{a_i}"] = distances[a_i]
    buyers.append(Buyer(f"b_{b_i}", distance_, beta=0.99, pv=(10+b_i*10), pvmax=(10+b_i*10)))
auctions = [Auction(f"a_{i}", prices[i]) for i in range(len(distances))]
for a in auctions:
    for b in buyers:
        a.add_buyer(b)
        b.inform_price(a.id, a.price)

In [ ]:
results = run_auctions(auctions, buyers, 100*1001)

In [ ]:
plot_dist_price_results(results, buyers)

In [ ]:
plot_utility(results, buyers, [0.99 for _ in range(9)])

In [ ]:
results